In [1]:
# Jupyter magic extensions that makes our live easier
%load_ext autoreload
%autoreload 2

In [47]:
from src.models.train_pipeline import train_pipeline
from src.models.models import SklearnAdaBoostModel
from src.data.make_dataset import PandasDatasetLoader

dataset_path = "data/raw"
labels_path = "data/labels/"

train_pipeline(PandasDatasetLoader(dataset_path, labels_path), SklearnAdaBoostModel())

n_labels:                                       filepath  label
986   1110/010/UTC-2020_03_06-02_40_00.parquet      0
987   1110/010/UTC-2020_03_06-02_50_00.parquet      0
988   1110/010/UTC-2020_03_06-03_00_00.parquet      0
989   1110/010/UTC-2020_03_06-03_10_00.parquet      0
990   1110/010/UTC-2020_03_06-03_20_00.parquet      0
...                                        ...    ...
2374  1110/016/UTC-2020_03_17-13_30_00.parquet      0
2375  1110/016/UTC-2020_03_17-13_40_00.parquet      0
2376  1110/016/UTC-2020_03_17-13_50_00.parquet      0
2377  1110/016/UTC-2020_03_17-14_00_00.parquet      0
2378  1110/016/UTC-2020_03_17-14_10_00.parquet      0

[1393 rows x 2 columns]
(1114, 11)
(279,)
Resultado AUC:  {'1110': 0.4981949458483754} 

Reporte:  {'1110': '              precision    recall  f1-score   support\n\n           0       0.99      1.00      0.99       277\n           1       0.00      0.00      0.00         2\n\n    accuracy                           0.99       279\n   macro

In [3]:
import os

print(os.getcwd())

os.chdir("..")

/home/jmjl/PycharmProjects/epilepsia_challenge/notebooks


In [4]:
import os

print(os.getcwd())

c:\Users\rober\Documents\Epilepsy_Challenge\epilepsia_challenge
